### Imports

In [16]:
import sys, re, os, shutil
import numpy as np
import skimage.io as io
import skimage.transform as trans
import matplotlib.image as mpimg

from PIL import Image ,ImageEnhance
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator

### Constants

In [5]:
SIDE = 256
DATA_ROOT = 'data/'
PATH_ROOT = ''

### Load Colab Notebook settings

**Link Google Drive**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
PATH_ROOT = 'gdrive/My Drive/'  #change dir to your project folder


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


**Try GPU power**

In [4]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7410 sha256=d2a73514576db0602c11882accbcd4d819cec20b1dcc7664a0075c70743694b3
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.5 GB  | Proc size: 591.9 MB
GPU RAM Free: 16015MB | Used: 265MB | Util   2% | Total 16280MB


In [0]:
#!kill -9 -1

## Train the model

In [2]:
# mettre dans un .py qaund on a trouvé les bons params

def sorted_aphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

def data_load(directory_name): 
    filenames = sorted_aphanumeric(os.listdir(directory_name))
    imgs = []
    for i, fileNb in enumerate(filenames):
        if(fileNb[0]!='.'):
            full_name = directory_name+fileNb
            img=mpimg.imread(full_name)
            imgr = img_to_array(img)
            imgs.append(imgr)
            sys.stdout.write("\rImage {}/{} is being loaded".format(i+1,len(filenames)))
            sys.stdout.flush()    
    return np.asarray(imgs)

# target_size = (256,256)
def dataGenerator(batch_size, train_path, image_folder, mask_folder, aug_dict, target_size,
                  image_color_mode = "rgb", mask_color_mode = "grayscale",
                  image_save_prefix  = "image",mask_save_prefix  = "mask",
                  flag_multi_class = False,num_class = 2,save_to_dir = None, seed = 1):

    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        if(np.max(img) > 1):
            img = img / 255
            mask = mask /255
            mask[mask > 0.5] = 1
            mask[mask <= 0.5] = 0
        yield (img,mask)

def uneti(input_size, pretrained_weights=None, verbose=True):
    inputs = Input(shape=input_size)

    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
   
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    conv5 = BatchNormalization()(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv5))
    merge6 = concatenate([conv4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    conv6 = BatchNormalization()(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    conv7 = BatchNormalization()(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    conv8 = BatchNormalization()(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)
    
    ##binary_crossentropy
    ##categorical_crossentropy
    model.compile(optimizer = Adam(lr = 0.0005), loss = 'binary_crossentropy', metrics = ['accuracy'])
    if(verbose == True):
        model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

**1. Split data into validation and training directories** [Local]

In [27]:
# reinit dir 
#TODO: cross validation over it
import shutil, os

def augment_saturation(img):
    converter = ImageEnhance.Color(img)
    return converter.enhance(3)

def create_validation_train_directory(path, dir_images, dir_labels, seed):                           
    origin_dirs = [dir_images, dir_labels]
    for name in origin_dirs:
        filenames = sorted_aphanumeric(os.listdir(path+name))
        if('.DS_Store' in filenames):
            filenames = np.delete(filenames, 0)
        np.random.seed(seed)
        permut = np.random.permutation(len(filenames))
        index = int(len(permut)*0.2)
        test_ind = permut[:index]
        test_filenames = filenames[test_ind]
        trai_ind = permut[index:]
        trai_filenames = filenames[trai_ind]
        new = [name+'_te', name+'_tr']
        for n in new:  
            if os.path.exists(path+n):
                shutil.rmtree(path+n)
            os.mkdir(path+n)
        for test_f in test_filenames:
            img = Image.open(path+name+'/'+test_f)
            #img = augment_saturation(img)
            img.save(path+new[0]+'/'+test_f)
        for trai_f in trai_filenames:
            img = Image.open(path+name+'/'+trai_f)
            #img = augment_saturation(img)
            img.save(path+new[1]+'/'+trai_f)

    
create_validation_train_directory(DATA_ROOT+'train/',
                                  'images',
                                  'groundtruth', 1)

**2. Data augmentation, generators, callbacks**

In [0]:
data_gen_args = dict(rotation_range=0.15,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')

train_generator = dataGenerator(2,
                                PATH_ROOT+DATA_ROOT+'/train',
                                'images_tr',
                                'groundtruth_tr'
                                ,data_gen_args,
                                (SIDE,SIDE))

validation_generator = dataGenerator(2,
                                PATH_ROOT+DATA_ROOT+'/train',
                                'images_te',
                                'groundtruth_te'
                                ,data_gen_args,
                                (SIDE,SIDE))

filepath = "weights.{epoch:02d}-{acc:.2f}.hdf5"
cp_callback = ModelCheckpoint(filepath=filepath, verbose=1, save_weights_only=True, period=1)

i = 0
for (img,mask) in myGene: 
    i = i+1
    print(i, img.shape, mask.shape)

**3. Load the model**

In [9]:
model = uneti((SIDE,SIDE,3), pretrained_weights=None, verbose=True)

















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       batch_normalizat

**4. Train the model**

In [10]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=2000,
                    epochs=10, 
                    verbose=1,
                    validation_data = validation_generator,
                    validation_steps = 700,
                    validation_freq=1,
                    initial_epoch=0,
                    callbacks=[cp_callback]
                    )



Epoch 1/10
Found 24 images belonging to 1 classes.
Found 24 images belonging to 1 classes.
Found 96 images belonging to 1 classes.
Found 96 images belonging to 1 classes.
2000/2000 [==============================] - 366s 183ms/step - loss: 0.2308 - acc: 0.9166 - val_loss: 0.1959 - val_acc: 0.9374

Epoch 00001: saving model to weights.01-0.92.hdf5
Epoch 2/10
2000/2000 [==============================] - 287s 143ms/step - loss: 0.1356 - acc: 0.9587 - val_loss: 0.1733 - val_acc: 0.9420

Epoch 00002: saving model to weights.02-0.96.hdf5
Epoch 3/10
2000/2000 [==============================] - 286s 143ms/step - loss: 0.0985 - acc: 0.9691 - val_loss: 0.1985 - val_acc: 0.9365

Epoch 00003: saving model to weights.03-0.97.hdf5
Epoch 4/10
2000/2000 [==============================] - 287s 143ms/step - loss: 0.0669 - acc: 0.9785 - val_loss: 0.1859 - val_acc: 0.9411

Epoch 00004: saving model to weights.04-0.98.hdf5
Epoch 5/10
2000/2000 [==============================] - 287s 144ms/step - loss: 0.

### Save model to disk

In [11]:
# serialize model to JSON
OUTPUT_FILENAME = "Model_UNET_256_acc98"

model_json = model.to_json()
with open(OUTPUT_FILENAME + ".json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights(OUTPUT_FILENAME + ".h5")
print("Saved model to disk")

Saved model to disk


____
____
____
### Predictions (don't run it on Colab)

**Load model** [Optional]

In [3]:
from keras.models import model_from_json
json_file = open('model_saved/Model_UNET_256_acc99_valid.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

**Resize images in a folder** [Optional]

In [6]:
from PIL import Image
import os
path1 = "data/tmp/"
path2 = "data/test/"

filenames = os.listdir(path1)
for i, fileNb in enumerate(filenames):
    if(fileNb!='.DS_Store'):        
        im1 = Image.open(path1+fileNb)
        # use one of these filter options to resize the image
        im2 = im1.resize((SIDE, SIDE), Image.NEAREST)
        im2.save(path2+fileNb)

In [8]:
data_test = data_load('data/test/')

test_datagenerator  = ImageDataGenerator()
testGene  = test_datagenerator.flow(data_test, batch_size=1)

model = uneti(input_size = (SIDE,SIDE,3), pretrained_weights=None, verbose=False)

model.load_weights("model_saved/Model_UNET_256_acc99_valid.h5")

results = model.predict(data_test,verbose=1)

50/50 [==============================] - 136s 3s/step


In [9]:
from skimage import img_as_ubyte

def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def saveResult(from_path, save_path, predictions, concat=True):
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    list_im_name = sorted_aphanumeric(os.listdir(from_path))
    if(list_im_name[0]=='.DS_Store'):
        list_im_name=list_im_name[1:]
    for name, pred in zip(list_im_name, predictions):
        img = mpimg.imread(from_path+name)
        if(concat):
            if(len(pred.shape)==3 and pred.shape[2]==1):
                pred = pred.reshape(pred.shape[0],pred.shape[1])
            cimg = concatenate_images(img, pred)
            io.imsave(save_path+"/prediction"+name[4:], cimg)
        else:
            io.imsave(save_path+"/prediction"+name[4:], pred)

In [11]:
saveResult("data/test/", "data/pred_unet_256_99_validb", results, concat=True)